In [8]:
!pip install gradio unidecode

In [14]:
!sudo apt-get install swi-prolog
print("Đã cài xong SWI-Prolog")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swi-prolog is already the newest version (8.4.2+dfsg-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Đã cài xong SWI-Prolog
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swi-prolog is already the newest version (8.4.2+dfsg-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Đã cài xong SWI-Prolog


In [21]:
%%writefile engine.pl
:- op(950, yfx, @).
:- op(900, yfx, x).

% --- REDUCTION DRIVER ---
reduce_fully(Term, Final) :-
    step(Term, Next), !, reduce_fully(Next, Final).
reduce_fully(Term, Term).

% --- STEPS ---
step(lambda(X, Body) @ Arg, Result) :- substitute(Body, X, Arg, Result).
step(drs(R1, C1) x drs(R2, C2), drs(R, C)) :- append(R1, R2, R), append(C1, C2, C).

% Traversal
step(A x B, ResA x B) :- step(A, ResA).
step(A x B, A x ResB) :- step(B, ResB).
step(A @ B, ResA @ B) :- step(A, ResA).
step(A @ B, A @ ResB) :- step(B, ResB).

% --- SUBSTITUTE ---
substitute(Term, Old, New, New) :- Term == Old, !.
substitute(Term, _, _, Term) :- var(Term), !.
substitute(Term, _, _, Term) :- atomic(Term), !.
substitute(Term, Old, New, Result) :-
    Term =.. [F | Args],
    maplist(substitute_helper(Old, New), Args, NewArgs),
    Result =.. [F | NewArgs].
substitute_helper(Old, New, Arg, Res) :- substitute(Arg, Old, New, Res).

Overwriting engine.pl


In [31]:
%%writefile facts.pl
:- discontiguous thich/2.
:- discontiguous co/2.
:- discontiguous nhan_vien/2.
:- discontiguous ban_dl/2.
:- discontiguous anh/1.
:- discontiguous la/2.
la(X, X).

% 1. THỰC THỂ
cong_ty(samsung).
tap_doan(apple).
dien_thoai(iphone).

% 2. GIỚI TÍNH
anh(tu).
anh(nghia).

% 3. QUAN HỆ
nhan_vien(tu, samsung).
nhan_vien(nghia, apple).
ban_dl(tu, nghia).
ghet(tu, nghia).
thich(tu, iphone).
co(nghia, iphone).

% 4. QUY TẮC SUY DIỄN
ban(X, Y) :- ban_dl(X, Y).
ban(X, Y) :- ban_dl(Y, X).
thich(X, iphone) :- co(X, iphone).

% 5. HỖ TRỢ CÂU HỎI
question_target(_).

Overwriting facts.pl


In [23]:
%%writefile grammar.pl
:- ensure_loaded('engine.pl').

% --- VĂN BẢN ---
txt(Sem) --> s(Sem).
txt(Sem) --> s(S1), txt(S2), { Sem = (S1 x S2) }.

% --- CÂU ---
s(Sem) --> np(NP), vp(VP), { Sem = (NP @ VP) }.

% --- CỤM DANH TỪ (NP) ---
% 1. Tên riêng & Từ hỏi
np(Sem) --> nnp(Sem).
np(Sem) --> wp_pronoun(Sem).

% 2. Mạo từ + Danh từ (VD: "một công ty")
np(Sem) --> det(Det), nn(N), { Sem = (Det @ N) }.

% 3. Danh từ + Từ hỏi (VD: "công ty gì")
np(Sem) --> nn(N), wp_det(Det), { Sem = (Det @ N) }.

% 4. Đồng sở chỉ (VD: "anh ấy")
np(Sem) --> nn_gender(N), det_dem(D), { Sem = (D @ N) }.

% 5. Quan hệ (VD: "nhân viên của Samsung")
np(Sem) --> nn_rel(Rel), prep(_), np(Obj), { Sem = (Rel @ Obj) }.
np(Sem) --> nn_rel(Rel), np(Obj), { Sem = (Rel @ Obj) }.


% --- CỤM ĐỘNG TỪ (VP) ---

% 1. Động từ thường + Tân ngữ (VD: "ghét Nghĩa")
vp(Sem) --> vb(V), np(Obj), { Sem = (V @ Obj) }.

% 2. Động từ "LÀ" + Cụm danh từ (VD: "là một tập đoàn")
% Trường hợp này Obj là một Lượng từ (Generalized Quantifier)
% Ta dùng vb_be_eq để tạo quan hệ X = Y
vp(Sem) --> vb_be_eq(Be), np(Obj), { Sem = (Be @ Obj) }.

% 3. Động từ "LÀ" + Danh từ chỉ loại (VD: "là tập đoàn") -> FIX LỖI CỦA BẠN
% Trường hợp này N là một Thuộc tính (Property/Predicate)
% Ta dùng vb_be_id (Identity) để giữ nguyên thuộc tính đó
vp(Sem) --> vb_be_id(Be), nn(N), { Sem = (Be @ N) }.


% ================= TỪ VỰNG =================

% --- TÊN RIÊNG ---
nnp(lambda(Q, drs([tu],[]) x (Q@tu))) --> [tu].
nnp(lambda(Q, drs([nghia],[]) x (Q@nghia))) --> [nghia].
nnp(lambda(Q, drs([samsung],[]) x (Q@samsung))) --> [samsung].
nnp(lambda(Q, drs([apple],[]) x (Q@apple))) --> [apple].
nnp(lambda(Q, drs([iphone],[]) x (Q@iphone))) --> [iphone].

% --- TỪ HỎI ---
wp_pronoun(lambda(Q, drs([X], [question_target(X)]) x (Q @ X))) --> [ai].
wp_pronoun(lambda(Q, drs([X], [question_target(X)]) x (Q @ X))) --> [gi].
wp_det(lambda(P, lambda(Q, drs([X], [question_target(X)]) x (P@X) x (Q@X)))) --> [gi].

% --- DANH TỪ CHUNG ---
nn(lambda(X, drs([], [cong_ty(X)]))) --> [cong_ty].
nn(lambda(X, drs([], [tap_doan(X)]))) --> [tap_doan].
nn(lambda(X, drs([], [dien_thoai(X)]))) --> [dien_thoai].

% --- DANH TỪ QUAN HỆ ---
nn_rel(lambda(ObjGQ, lambda(SubjProp,
    ObjGQ @ lambda(Y, drs([X], [nhan_vien(X,Y)]) x (SubjProp @ X))
))) --> [nhan_vien].

nn_rel(lambda(ObjGQ, lambda(SubjProp,
    ObjGQ @ lambda(Y, drs([X], [ban(X,Y)]) x (SubjProp @ X))
))) --> [ban].

% --- ĐỘNG TỪ ---
vb(lambda(ObjGQ, lambda(S, ObjGQ @ lambda(O, drs([], [ghet(S,O)]))))) --> [ghet].
vb(lambda(ObjGQ, lambda(S, ObjGQ @ lambda(O, drs([], [thich(S,O)]))))) --> [thich].
vb(lambda(ObjGQ, lambda(S, ObjGQ @ lambda(O, drs([], [co(S,O)]))))) --> [co].

% --- ĐỘNG TỪ "LÀ" (QUAN TRỌNG) ---

% Cách 1: Hàm đồng nhất (Identity: lambda P.P)
% Dùng cho: "Samsung là tập đoàn" (Samsung có thuộc tính tập đoàn)
vb_be_id(lambda(P, P)) --> [la].

% Cách 2: Quan hệ bằng (Equality: X = Y)
% Dùng cho: "Samsung là một tập đoàn" (Tồn tại X là tập đoàn và Samsung = X)
% Logic: lambda(GQ, lambda(S, GQ @ lambda(O, drs([], [S=O]))))
vb_be_eq(lambda(ObjGQ, lambda(S, ObjGQ @ lambda(O, drs([], [S=O]))))) --> [la].


% --- MẠO TỪ ---
% "Một" là lượng từ tồn tại
det(lambda(P, lambda(Q, drs([Z], []) x (P@Z) x (Q@Z)))) --> [mot].

% --- HƯ TỪ KHÁC ---
nn_gender(lambda(X, drs([], [anh(X)]))) --> [anh].
det_dem(lambda(P, lambda(Q, drs([X],[]) x (P@X) x (Q@X)))) --> [ay].
prep(lambda(P, P)) --> [cua].

Overwriting grammar.pl


In [24]:
%%writefile test.pl
:- ensure_loaded('engine.pl').
:- ensure_loaded('grammar.pl').
:- ensure_loaded('facts.pl').

% --- [QUAN TRỌNG] KHAI BÁO TOÁN TỬ & ---
% Định nghĩa & là toán tử kết hợp phải (xfy) với độ ưu tiên 800
:- op(800, xfy, &).

% --- 1. HÀM CHUYỂN ĐỔI DRS SANG FOL ---

% Hàm phụ: Chuyển danh sách điều kiện thành chuỗi logic A & B & C
list_to_and([], true).          % Rỗng -> True
list_to_and([One], One) :- !.   % 1 phần tử -> Chính nó
list_to_and([Head|Tail], Head & Rest) :- % Nhiều phần tử -> Head & Rest
    list_to_and(Tail, Rest).

% Hàm chính: Chuyển đổi cấu trúc DRS -> FOL
% Trường hợp 1: Có biến (Refs khác rỗng) -> Thêm lượng từ tồn tại (exists)
drs_to_fol(drs(Refs, Conds), exists(Refs, Body)) :-
    Refs \= [], !,
    list_to_and(Conds, Body).

% Trường hợp 2: Không có biến (Refs rỗng) -> Chỉ nối các điều kiện
drs_to_fol(drs([], Conds), Body) :-
    list_to_and(Conds, Body).

% Catch-all: Nếu lỗi cấu trúc, trả về fail để in thông báo lỗi
drs_to_fol(_, _) :- fail.


% --- 2. BỘ GIẢI (SOLVER) ---
solve_drs(drs(_, Conds), TargetVars) :-
    maplist(call, Conds),
    findall(X, (member(question_target(X), Conds)), TargetVars).


% --- 3. HÀM XỬ LÝ CHÍNH (PROCESS) ---
process_question(ListTu) :-
    (   txt(LambdaRaw, ListTu, []),
        reduce_fully(LambdaRaw, FinalDRS)
    ->
        % A. IN DRS
        write('DRS: '), write(FinalDRS), nl,

        % B. IN FOL (Đã an toàn hơn)
        (   drs_to_fol(FinalDRS, FolForm)
        ->  write('FOL: '), write(FolForm), nl
        ;   write('FOL: error_converting_structure'), nl
        ),

        % C. TRẢ LỜI
        (   solve_drs(FinalDRS, Targets),
            ( Targets \= [] ->
                sort(Targets, UniqueTargets),
                write('RESULT: '), write(UniqueTargets)
            ;
                write('RESULT: YES')
            )
        ->  true
        ;   write('RESULT: NO')
        )
    ;   write('RESULT: ERROR (Khong hieu cau hoi)')
    ),
    halt.

Overwriting test.pl


In [ ]:
import gradio as gr
from unidecode import unidecode
import subprocess
import re
import os
from openai import OpenAI


os.environ["GROQ_API_KEY"] = "" #chèn link api key

client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1",
)

MODEL_NAME = "llama-3.3-70b-versatile"

def get_knowledge_base():
    try:
        with open("facts.pl", "r", encoding="utf-8") as f:
            return f.read()
    except:
        return "anh(tu). cong_ty(samsung). nhan_vien(tu, samsung)."


def ask_llm_direct(question):
    kb = get_knowledge_base()

    prompt = f"""
    Bạn là một máy suy luận logic thuần túy.
    Nhiệm vụ: Dựa vào Knowledge Base (KB) cung cấp dưới đây, hãy chuyển câu hỏi sang Logic (FOL/Prolog) và tìm đáp án.

    === KNOWLEDGE BASE ===
    {kb}
    ======================

    Câu hỏi: "{question}"

    Yêu cầu bắt buộc:
    1. Tuyệt đối KHÔNG giải thích, KHÔNG nói lời dẫn.
    2. Chỉ trả về đúng 2 dòng theo định dạng sau:

    FOL: <biểu thức logic hoặc prolog query tương ứng>
    ANSWER: <YES / NO / Danh sách kết quả dạng [a, b]>

    3. QUY TẮC QUAN TRỌNG VỚI TỪ "LÀ":
       - Với câu dạng "A là B" (định danh/phân loại), hãy dùng vị từ một ngôi B(A).
       - Ví dụ: "Samsung là tập đoàn" => tap_doan(samsung).
       - TUYỆT ĐỐI KHÔNG VIẾT: la(samsung, tap_doan).
    """

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=150
        )
        content = response.choices[0].message.content.strip()
        return "🤖 LLM (Direct Logic):\n" + content

    except Exception as e:
        return f"Lỗi gọi LLM: {str(e)}"

COMPOUND_WORDS = {
    "công ty": "cong_ty", "tập đoàn": "tap_doan", "điện thoại": "dien_thoai",
    "nhân viên": "nhan_vien", "bạn bè": "ban"
}
STOP_WORDS = ["phai khong", "khong", "ha", "nhi", "a", "u", "chua", "roi"]
MAIN_VERBS = ["ghet", "thich", "yeu", "la"]

def preprocess_text(text):
    text_no_accent = unidecode(text).lower()
    text_clean = re.sub(r'[^\w\s]', '', text_no_accent)
    for word, replacement in COMPOUND_WORDS.items():
        text_clean = text_clean.replace(unidecode(word).lower(), replacement)
    for stop_word in STOP_WORDS:
        text_clean = re.sub(r'\b' + stop_word + r'\b', '', text_clean)
    tokens = text_clean.split()
    final_tokens = []
    for i, token in enumerate(tokens):
        if token == "co":
            if i + 1 < len(tokens) and tokens[i+1] in MAIN_VERBS: continue
        final_tokens.append(token)
    return "[" + ", ".join(final_tokens) + "]"

def toString(d):
    OPS = ["&", "v", "=>", "<=>", "@", "x"]
    if type(d) is dict:
        functor = d.get("functor")
        args = d.get("args")
        if functor in OPS:
            st = [toString(args[0]), "{}".format(functor)]
            for e in args[1:]: st.append(toString(e))
            return " ".join(st)
        elif functor == "exists":
            return f"∃{args[0]} ({toString(args[1])})"
        else:
            st = ["{}({}".format(functor, toString(args[0]))]
            for e in args[1:]: st.append("{}".format(toString(e)))
            return ",".join(st) + ")"
    return "{}".format(d)

def parse_fol_string(text):
    text = text.strip()
    exists_match = re.match(r'exists\(([\w\[\],_ ]+),\s*(.*)\)$', text)
    if exists_match:
        return {"functor": "exists", "args": [exists_match.group(1).strip(), parse_fol_string(exists_match.group(2).strip())]}
    if '&' in text:
        return {"functor": "&", "args": [parse_fol_string(p) for p in text.split('&')]}
    match = re.match(r'(\w+)\((.*)\)', text)
    if match:
        return {"functor": match.group(1), "args": [a.strip() for a in match.group(2).split(',')]}
    return text

def ask_prolog_pipeline(question):
    try:
        prolog_list = preprocess_text(question)
        print(f"DEBUG Input: {prolog_list}")
        command = f"swipl -s test.pl -g \"process_question({prolog_list})\""
        result = subprocess.run(command, shell=True, capture_output=True, text=True)

        drs_match = re.search(r'DRS: (.*)', result.stdout)
        fol_match = re.search(r'FOL: (.*)', result.stdout)
        ans_match = re.search(r'RESULT: (.*)', result.stdout)

        drs_text = drs_match.group(1).strip() if drs_match else "..."
        fol_raw = fol_match.group(1).strip() if fol_match else "..."

        try:
            if fol_raw != "..." and "error" not in fol_raw:
                fol_pretty = toString(parse_fol_string(fol_raw))
            else: fol_pretty = fol_raw
        except: fol_pretty = fol_raw

        if ans_match:
            ans = ans_match.group(1).strip()
            final_ans = "YES" if ans == "YES" else ("NO" if ans == "NO" else ans)
            return (f"Đáp án: {final_ans}\n\n"
                    f"DRS Raw:\n{drs_text}\n\n"
                    f"FOL Structure:\n{fol_pretty}")
        else:
            return f"⚠️ Lỗi xử lý: {result.stdout}"
    except Exception as e: return f"Lỗi: {str(e)}"

def main_handler(question, mode):
    if mode == "Prolog Pipeline":
        return ask_prolog_pipeline(question)
    else:
        return ask_llm_direct(question)

with gr.Blocks(title="Computational Semantics Demo") as demo:
    gr.Markdown("# 🤖 Hệ Thống Hỏi Đáp: Logic vs LLM")
    gr.Markdown("Chọn phương pháp xử lý bên dưới:")

    with gr.Row():
        with gr.Column(scale=1):
            radio = gr.Radio(
                ["Prolog Pipeline", "LLM"],
                label="Chế độ xử lý",
                value="Prolog Pipeline"
            )
            inp = gr.Textbox(label="Câu hỏi", placeholder="Ví dụ: Ai là nhân viên của Samsung?")
            btn = gr.Button("Gửi câu hỏi", variant="primary")

        with gr.Column(scale=1):
            out = gr.Textbox(label="Kết quả chi tiết", lines=12)

    btn.click(fn=main_handler, inputs=[inp, radio], outputs=out)

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://66b7033ece72eda2f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


DEBUG Input: [tu, la, nhan_vien, cua, samsung]
DEBUG Input: [tu, la, nhan_vien, cua, samsung]
DEBUG Input: [tu, la, nhan_vien, cua, samsung, anh, ay, ghet, nghia]
DEBUG Input: [tu, la, nhan_vien, cua, samsung, anh, ay, ghet, nghia]
DEBUG Input: [tu, la, nhan_vien, cua, samsung]
